# Занятие 4: основы PyTorch, продолжение

### Практикум на ЭВМ для 317 группы, ВМК МГУ, кафедра ММП

### Варламова Арина, кафедра ММП ВМК МГУ


In [12]:
import torch
import numpy as np

## Вычисления на GPU

In [13]:
torch.cuda.is_available()

True

In [14]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


In [15]:
a = torch.ones(5)
print(a)
a = a.to(device)
print(a)

tensor([1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1.], device='cuda:0')


In [16]:
a = torch.ones(5)
print(a)
a = a.cuda()
print(a)

tensor([1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1.], device='cuda:0')


In [17]:
a = torch.ones(5, device='cuda:0')
print(a)
a = a.cpu()
print(a)

tensor([1., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 1., 1., 1., 1.])


## Перевод в numpy

In [18]:
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]


In [19]:
a.add_(1)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


In [20]:
a = np.ones(5)
b = torch.from_numpy(a)
print(a)
print(b)

[1. 1. 1. 1. 1.]
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [21]:
a += 1
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


In [22]:
a = torch.ones(5, device=device)
print(a)

b = a.numpy() # Must fail
print(b)

tensor([1., 1., 1., 1., 1.], device='cuda:0')


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

## Графы вычислений, автоматическое дифференцирование

In [23]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

### Производя операции с переменными, по которым нужно считать градиенты, мы конструируем граф вычислений:

In [24]:
Q = 3 * a**3 - b**2
Q.requires_grad

True

### В каждой переменной есть информация о том, как именно она была получена при проходе вперёд:

In [25]:
Q.grad_fn

### Проход назад выполняется с помощью вызова метода .backward():

In [26]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)
# Или Q.sum().backward()

### Вычисленные градиенты хранятся в a.grad и b.grad (справедливо только для листовых переменных графа, у которых reqieres_grad == True)

In [27]:
print(9 * a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


In [28]:
Q.grad

<ipython-input-28-fcbfbc63cb8e>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  Q.grad


### Можно считать граф для не листовых переменных, для этого необходимо воспользоваться методом .retain_graph:

In [29]:
x = torch.rand((3, 3), requires_grad=True)
y = x + 2
y.retain_grad()

z = torch.mean(y)
z.backward()
y.grad

tensor([[0.1111, 0.1111, 0.1111],
        [0.1111, 0.1111, 0.1111],
        [0.1111, 0.1111, 0.1111]])

### Можно вычислять производную по направлению:

In [30]:
x = torch.Tensor([1, 2, 3])
x.requires_grad = True

y = x * 2

gradients = torch.FloatTensor([0, 1, 0.0001])
y.backward(gradients)
print(x.grad)

tensor([0.0000e+00, 2.0000e+00, 2.0000e-04])


### С помощью специального мененджера контекста, можно не считать градиенты при вычислениях (например, для ускорения работы):

In [31]:
x = torch.Tensor([1, 2, 3])
x.requires_grad = True

with torch.no_grad():
    y = x * 2
    
y.grad_fn

In [32]:
x = torch.rand((3, 3), requires_grad=True)
x.numpy() # Must fail

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [36]:
x = torch.rand((3, 3), requires_grad=True)
x.detach().numpy() 

array([[0.5793321 , 0.2560078 , 0.16767746],
       [0.31539023, 0.08988065, 0.1116097 ],
       [0.34372973, 0.76385254, 0.5091339 ]], dtype=float32)